In [199]:
import os
import ee
import geemap

In [200]:
Map = geemap.Map()
Map


Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [201]:
geemap.ee_search(asset_limit=1000)

In [203]:
lenaPolygon = ee.FeatureCollection('users/reaganokoth004/LenaPolygon')
#Map.addLayer(lenaPolygon, {}, "LenaPolygon")

In [204]:
#Downloads the html files of the Doc, Assets, Script in earth engine and 
#allows you to search and import them (and the codes) to notebook 
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_csv = os.path.join(out_dir, 'ee_api.csv')
geemap.ee_api_to_csv(out_csv)

In [205]:
ROI = (lenaPolygon)

In [206]:
def maskClouds(image):
  # Bits 3 and 5 are cloud shadow and cloud, respectively.
  cloudShadowBitMask = 1 << 3
  cloudsBitMask = 1 << 5

  # Get the pixel QA band.
  qa = image.select('pixel_qa')

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
      .And(qa.bitwiseAnd(cloudsBitMask).eq(0))

  # Return the masked image, scaled to reflectance, without the QA bands.
  return image.updateMask(mask).divide(10000) \
      .select("B[0-9]*") \
      .copyProperties(image, ["system:time_start"])
# }

In [207]:
#import Landsat 8 images between june and August 2018 falling within ROI
images2018 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
         .filterBounds(ROI).filterDate('2018-06-01', '2018-08-31').map(maskClouds)


In [208]:
# Compute the median in each band, each pixel.
# Band names are B1_median, B2_median, etc.
composite_median = images2018.reduce(ee.Reducer.median()).clip (ROI)

In [209]:
# The output is an Image.  Add it to the map.
vis_param = {'bands': ['B4_median', 'B3_median', 'B2_median'], 'gamma': 1.6}
Map.centerObject(ROI,8)
Map.addLayer(composite_median, vis_param, 'Median')


# Display the map.
Map

Map(bottom=1826.0, center=[72.98881183792554, 126.34373575217435], controls=(WidgetControl(options=['position'…

In [210]:
#Calculate ndvi of a single image composite and clip it to ROI
ndvi = composite_median.normalizedDifference(['B5_median', 'B4_median']).clip(ROI)

In [211]:
#ndvi visualization parameters

palette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']

ndvi_parameters = {'min': 0,
                   'max': 1,
                   'palette': palette,
                   'region': ROI}

In [212]:

#add the NDVI image to the console
Map.addLayer(ndvi, ndvi_parameters, "NDVI")
Map

Map(bottom=13224.0, center=[72.98881183792554, 126.34373575217435], controls=(WidgetControl(options=['position…

In [213]:
# Use these bands for prediction.
bands = ['B1_median','B2_median', 'B3_median', 'B4_median', 'B5_median', 'B6_median', 'B7_median']

In [214]:
TrainingData = ee.FeatureCollection('users/reaganokoth004/TrainingData')
Map.addLayer(TrainingData, {}, "TrainingData")

In [215]:
# This property of the table stores the land cover labels.
label = 'landcover'

# Overlay the points on the imagery to get training.
training = composite_median.select(bands).sampleRegions(**{
  'collection': TrainingData,
  'properties': [label],
  'scale': 30
})

In [216]:
# Train a CART classifier with default parameters.
trained = ee.Classifier.cart().train(training, label, bands)

In [217]:
# Classify the image with the same bands used for training.
classified = composite_median.select(bands).classify(trained).clip(ROI)

In [218]:
Map.addLayer(classified,
             {'min': 0, 'max': 7, 
              'palette': ['#206f1b', 
                          '#777957', 
                          '#44a243', 
                          '#208c73', 
                          '#bb7936', 
                          '#942a6e', 
                          '#f7ff22', 
                          '#202cff']},
             'classes')


Map


Map(bottom=13224.0, center=[72.98881183792554, 126.34373575217435], controls=(WidgetControl(options=['position…

In [160]:
# "map" the NDVI band over image collection
#image = ee.Algorithms.Landsat.simpleComposite(**{
#ndvi2018 = images2018_masked.map(getNDVI);
#ndvi2018 = image.map(getNDVI)
#print(ee.Image(ndvi2018.first()))

In [165]:
#composite the image based on max NDVI
#this code creats a single image composit based on max NDVI value
#composite = ndvi2018.qualityMosaic('NDVI').clip(ROI);

In [159]:
#Visualization params for NDVI
#ndviPalette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
 #              '74A901', '66A000', '529400', '3E8601', '207401', '056201',
 #              '004C00', '023B01', '012E01', '011D01', '011301'];
               

In [158]:
#Calculate ndvi (develop ndvi function) that can be mapped on multiple images

#def getNDVI(img):
 # return img.addBands(img.normalizedDifference(ee.List(['B5', 'B4'])).rename('NDVI'))

In [156]:
#make a composite from the image collection using landsat simple composite algorithm,
#filter the images by date and apply cloudmaks to the composite
#image = ee.Algorithms.Landsat.simpleComposite(**{
#  'collection': images2018,
#  'asFloat': True
#})
#print (image)

In [157]:

# apply the masking function on the image collection
#images2018_masked = images2018.map(maskClouds)

# visualize the first image in the collection, pre- and post- mask
#visParams = {
#    'bands': ['B4','B3','B2'], 
 #   'min': 0, 
  #  'max': 3000}

#Map.addLayer(ee.Image(images2018_masked.first()), visParams, 'cloudsmasked_2018', False)
#Map.addLayer(ee.Image(images2018.first()), visParams, 'original_2018', False)

In [173]:
#areaImage = ee.Image.pixelArea().addBands(
#      classified)

In [178]:
#classImage = ee.Image(classified).select('landcover')
#region = ee.Geometry.Point(lenaPolygon)

In [188]:
#class_areas = ee.Image.pixelArea().addBands(classImage).reduceRegion(**
    {
 #     'reducer': ee.Reducer.sum().group(**{
  #    'groupField': 1,
 #     'groupName': 'landcover',
  #  }),
  #  'geometry': region,
  #  'scale': 1,  # sample the geometry at 1m intervals
 #   'maxPixels': 1e10
  #}).get('groups');



IndentationError: unindent does not match any outer indentation level (<tokenize>, line 8)